# Analysis of Demographic Parity and Equalized Odds

This is a notebook I created while reading [Equality of Opportunity in Supervised Learning](https://arxiv.org/abs/1610.02413). 

I am not sure there is a way to access the code the authors used to produce their plots/results, however, after some googling I found the [Fairlearn](https://fairlearn.github.io/v0.5.0/index.html#) package, which seemingly aims to _empowers developers of artificial intelligence (AI) systems to assess their system's fairness and mitigate any observed unfairness issues_. Reading their documentation they seem to implement, among others, the methodology described by the paper above.

I thought it would be good to play around with this tool to assimilate the theoretical concepts introduced by the paper, so what you find below is a minimal overview of the practical use of this methodology, compared to another standard definition of fairness called _Demographic Parity_ (also implemented in Fairlearn)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [ ]:
data = fetch_openml(data_id=1590, as_frame=True)
X = pd.get_dummies(data.data)
y_true = (data.target == '>50K') * 1
sex = data.data['sex']
sex.value_counts()

In [ ]:
from fairlearn.metrics import MetricFrame
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier

In [ ]:
classifier = DecisionTreeClassifier(min_samples_leaf=10, max_depth=4)
classifier.fit(X, y_true)
y_pred = classifier.predict(X)

### No fairness measure

In [ ]:
gm = MetricFrame(accuracy_score, y_true, y_pred, sensitive_features=sex)
print(f"Total accuracy score: {gm.overall}\n")
print(f"Accuracy score by: {gm.by_group}\n")

### Fairness metrics

When working on fairness, before we can start _fairifying_ models we need to establish fairness metrics to optimise. Here we consider two definitions _Demographic Parity_ and _Equalized Odds_.

* **Demographic Parity**: is defined as `Pr(Y_pred=1 | A=1) = Pr(Y_pred=1 | A=0)`
* **Equalized Odds**: is defined as `Pr(Y_pred=1 | A=1, Y=y) = Pr(Y_pred=1 | A=0, Y=y)`

Since in both cases we won't get perfect equality a good way to assess fairness is to look at the difference between the terms, and this is exactly what is done in `fairnlearn`:

For _Demographic Parity_ we have:
```
demographic_parity_difference = |Pr(Y_pred=1 | A=1) - Pr(Y_pred=1 | A=0)|
```

For _Equalized Odds_ we have:
```
equalized_odds_difference = max(
    |Pr(Y_pred=1 | A=1, Y=0) - Pr(Y_pred=1 | A=0, Y=0)|,
    |Pr(Y_pred=1 | A=1, Y=1) - Pr(Y_pred=1 | A=0, Y=1)|
)
```

In [ ]:
from fairlearn.metrics import demographic_parity_difference, equalized_odds_difference

dp_difference = demographic_parity_difference(y_true, y_pred, sensitive_features=sex)
print(f"Demographic parity difference: {dp_difference}")

eo_difference = equalized_odds_difference(y_true, y_pred, sensitive_features=sex)
print(f"Equalized odds difference: {eo_difference}")

### Demographic Parity

In [ ]:
from fairlearn.reductions import ExponentiatedGradient, DemographicParity

mitigator = ExponentiatedGradient(classifier, DemographicParity())
mitigator.fit(X, y_true, sensitive_features=sex)
y_pred_mitigated = mitigator.predict(X)

In [ ]:
gm_dp = MetricFrame(accuracy_score, y_true, y_pred_mitigated, sensitive_features=sex)
print(f"Total accuracy score {gm_dp.overall}")
print(gm_dp.by_group)

In [ ]:
dp_difference = demographic_parity_difference(y_true, y_pred_mitigated, sensitive_features=sex)
print(f"Demographic parity difference: {dp_difference}")

### Equalized Odds

In [ ]:
from fairlearn.postprocessing import ThresholdOptimizer
from fairlearn.reductions import EqualizedOdds

mitigator = ThresholdOptimizer(estimator=classifier, constraints='equalized_odds')
mitigator.fit(X, y_true, sensitive_features=sex)
y_pred_mitigated = mitigator.predict(X, sensitive_features=sex)

In [ ]:
gm_eo = MetricFrame(accuracy_score, y_true, y_pred_mitigated, sensitive_features=sex)
print(f"Total accuracy score {gm_eo.overall}")
print(gm_eo.by_group)

In [ ]:
eo_difference = equalized_odds_difference(y_true, y_pred_mitigated, sensitive_features=sex)
print(f"Equalized odds difference: {eo_difference}")